# Вычисление WER для TTS модели

**Примечание:** WER (Word Error Rate) обычно используется для оценки ASR моделей, но по требованию задания мы вычисляем WER для TTS модели следующим образом:
1. Синтезируем аудио из текста через TTS модель
2. Распознаем синтезированное аудио через ASR модель (Whisper)
3. Сравниваем оригинальный текст с распознанным текстом через WER

Это позволяет оценить качество синтеза речи косвенно - если синтезированное аудио распознается с низким WER, значит синтез качественный.


## Импорты и настройка


In [ ]:
import os
import json
import glob
import wave
import shutil
from pathlib import Path
from typing import List, Optional

import numpy as np
from tqdm.auto import tqdm

print("✓ Базовые импорты загружены")


## Установка зависимостей


In [ ]:
# Установка Whisper для ASR (если не установлен)
print("="*60)
print("УСТАНОВКА И ПРОВЕРКА ЗАВИСИМОСТЕЙ")
print("="*60)

try:
    import whisper
    print("✓ Whisper уже установлен")
except ImportError:
    print("Установка Whisper...")
    %pip install openai-whisper -q
    import whisper
    print("✓ Whisper установлен")

try:
    from piper import PiperVoice
    print("✓ Piper уже установлен")
except ImportError:
    print("Установка Piper...")
    %pip install piper-tts -q
    from piper import PiperVoice
    print("✓ Piper установлен")

try:
    from jiwer import wer
    print("✓ jiwer уже установлен")
except ImportError:
    print("Установка jiwer...")
    %pip install jiwer -q
    from jiwer import wer
    print("✓ jiwer установлен")

print("\n✓ Все зависимости готовы")


## Поиск TTS модели и конфигурации


In [ ]:
print("="*60)
print("ПОИСК TTS МОДЕЛИ")
print("="*60)

# Определяем корневую директорию (где находится этот ноутбук)
# В Jupyter ноутбуке используем текущую рабочую директорию
NOTEBOOK_DIR = os.getcwd()

print(f"Корневая директория: {NOTEBOOK_DIR}")

# Возможные места для поиска модели
# ПРИОРИТЕТ: сначала ищем в tts_data (обученная модель), потом в examples (примеры от преподавателя)
possible_model_paths = [
    # Сначала ищем в tts_data (обученная модель)
    os.path.join(NOTEBOOK_DIR, "tts_data", "final_model", "final_model.onnx"),
    os.path.join(NOTEBOOK_DIR, "tts_data", "**", "*.onnx"),
    # Потом в examples (примеры от преподавателя)
    os.path.join(NOTEBOOK_DIR, "examples", "nataha_ruslan.onnx"),
    os.path.join(NOTEBOOK_DIR, "examples", "*.onnx"),
]

# Ищем ONNX модель
onnx_model_path = None
onnx_config_path = None

for path_pattern in possible_model_paths:
    if '*' in path_pattern:
        # Глоб паттерн
        matches = glob.glob(path_pattern, recursive=True)
        if matches:
            # Берем первый найденный
            onnx_model_path = matches[0]
            print(f"✓ Найдена модель (glob): {onnx_model_path}")
            break
    else:
        # Прямой путь
        if os.path.exists(path_pattern):
            onnx_model_path = path_pattern
            print(f"✓ Найдена модель: {onnx_model_path}")
            break

if not onnx_model_path:
    print("⚠️  ONNX модель не найдена автоматически")
    print("\nПроверяемые пути:")
    for path in possible_model_paths:
        exists = "✓" if os.path.exists(path) else "✗"
        print(f"  {exists} {path}")
    print("\nПожалуйста, укажите путь к модели вручную в следующей ячейке")
else:
    # Ищем конфигурационный файл
    # ПРИОРИТЕТ: сначала ищем рядом с моделью, потом в tts_data, потом в examples
    config_candidates = [
        onnx_model_path + ".json",  # Рядом с моделью
        os.path.join(os.path.dirname(onnx_model_path), "nata_config.json"),
        os.path.join(NOTEBOOK_DIR, "tts_data", "nata_config.json"),  # В tts_data
        os.path.join(NOTEBOOK_DIR, "examples", "nata_config.json"),  # В examples (примеры)
    ]
    
    for config_path in config_candidates:
        if os.path.exists(config_path):
            onnx_config_path = config_path
            print(f"✓ Найден конфиг: {onnx_config_path}")
            break
    
    if not onnx_config_path:
        print("⚠️  Конфигурационный файл не найден, модель может загрузиться без него")

print(f"\n{'='*60}")


In [ ]:
print("="*60)
print("ЗАГРУЗКА TTS МОДЕЛИ")
print("="*60)

if not onnx_model_path:
    print("⚠️  Модель не найдена. Пожалуйста, укажите путь вручную:")
    print("   onnx_model_path = '/path/to/model.onnx'")
    print("   onnx_config_path = '/path/to/model.onnx.json'  # опционально")
    voice = None
else:
    try:
        print(f"Загрузка модели: {onnx_model_path}")
        if onnx_config_path:
            print(f"Использование конфига: {onnx_config_path}")
            voice = PiperVoice.load(onnx_model_path, config_path=onnx_config_path, use_cuda=True)
        else:
            print("Загрузка без конфига...")
            voice = PiperVoice.load(onnx_model_path, use_cuda=True)
        print("✓ TTS модель успешно загружена")
    except Exception as e:
        print(f"✗ Ошибка при загрузке модели: {e}")
        import traceback
        traceback.print_exc()
        voice = None

print(f"\n{'='*60}")


## Инициализация ASR модели (Whisper)


In [ ]:
print("="*60)
print("ИНИЦИАЛИЗАЦИЯ ASR МОДЕЛИ (WHISPER)")
print("="*60)

print("Загрузка модели Whisper для распознавания...")
try:
    asr_model = whisper.load_model("base")  # Используем base для баланса скорости и качества
    print("✓ Модель Whisper загружена")
except Exception as e:
    print(f"✗ Ошибка при загрузке Whisper: {e}")
    import traceback
    traceback.print_exc()
    asr_model = None

# Функция для распознавания аудио
def transcribe_audio_with_whisper(audio_path: str) -> str:
    """Распознает аудио через Whisper."""
    if asr_model is None:
        return ""
    
    try:
        result = asr_model.transcribe(audio_path, language="ru")
        return result["text"].strip()
    except Exception as e:
        print(f"Ошибка при распознавании {audio_path}: {e}")
        return ""

print("\n✓ ASR модель готова к использованию")
print(f"\n{'='*60}")


## Подготовка тестовых данных


In [ ]:
print("="*60)
print("ПОДГОТОВКА ТЕСТОВЫХ ДАННЫХ")
print("="*60)

# Функция для вычисления WER
def calculate_wer(true_text: str, predicted_text: str) -> float:
    """Вычисляет Word Error Rate между двумя текстами."""
    try:
        from jiwer import wer
        return wer(true_text, predicted_text)
    except:
        # Простая реализация, если jiwer не работает
        true_words = true_text.lower().split()
        pred_words = predicted_text.lower().split()
        
        if len(true_words) == 0:
            return 1.0 if len(pred_words) > 0 else 0.0
        
        errors = sum(1 for t, p in zip(true_words, pred_words) if t != p)
        errors += abs(len(true_words) - len(pred_words))
        return errors / len(true_words)

# Тестовые тексты для синтеза
# Можно заменить на загрузку из файла или использовать свои
test_texts = [
    "Привет, как дела?",
    "Распознавание и синтез речи это интересная область.",
    "Сегодня хорошая погода.",
    "Машинное обучение помогает создавать умные системы.",
    "Нейронные сети используются для обработки естественного языка.",
    "Технологии искусственного интеллекта развиваются очень быстро.",
    "Голосовые помощники становятся все более популярными.",
    "Синтез речи позволяет компьютерам говорить как люди.",
    "Качество синтеза речи постоянно улучшается.",
    "Современные модели могут генерировать очень естественную речь.",
]

# Пробуем найти тестовые данные из CSV файлов
possible_csv_paths = [
    os.path.join(NOTEBOOK_DIR, "tts_data", "valid.csv"),
    os.path.join(NOTEBOOK_DIR, "tts_data", "test.csv"),
    os.path.join(NOTEBOOK_DIR, "**", "valid.csv"),
    os.path.join(NOTEBOOK_DIR, "**", "test.csv"),
]

csv_data = []
csv_path = None

for path_pattern in possible_csv_paths:
    if '*' in path_pattern:
        matches = glob.glob(path_pattern, recursive=True)
        if matches:
            csv_path = matches[0]
            break
    else:
        if os.path.exists(path_pattern):
            csv_path = path_pattern
            break

if csv_path:
    print(f"✓ Найден CSV файл: {csv_path}")
    print("Загрузка данных из CSV...")
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if '|' in line:
                    # Формат: audio_file|text
                    parts = line.split('|', 1)
                    if len(parts) == 2:
                        csv_data.append(parts[1])  # Текст
        print(f"✓ Загружено {len(csv_data)} текстов из CSV")
        if len(csv_data) > 0:
            # Используем данные из CSV, ограничиваем до 50 примеров
            test_texts = csv_data[:50]
            print(f"Используем {len(test_texts)} примеров из CSV")
    except Exception as e:
        print(f"⚠️  Ошибка при чтении CSV: {e}")
        print("Используем стандартные тестовые тексты")
else:
    print("⚠️  CSV файл не найден, используем стандартные тестовые тексты")
    print(f"Количество тестовых текстов: {len(test_texts)}")

print(f"\nИтого тестовых примеров: {len(test_texts)}")
print(f"\n{'='*60}")


In [ ]:
print("="*60)
print("ВЫЧИСЛЕНИЕ WER ДЛЯ TTS МОДЕЛИ")
print("="*60)

# Проверяем, что модели загружены
if not voice:
    print("⚠️  TTS модель не загружена. Невозможно вычислить WER.")
    print("   Убедитесь, что модель найдена и загружена в предыдущих ячейках")
elif asr_model is None:
    print("⚠️  ASR модель не загружена. Невозможно вычислить WER.")
    print("   Убедитесь, что Whisper загружен в предыдущих ячейках")
else:
    print("\n✓ Обе модели загружены")
    print(f"Обрабатываем {len(test_texts)} примеров...")
    
    wers = []
    original_texts = []
    recognized_texts = []
    processed_count = 0
    errors_count = 0
    
    # Создаем временную директорию для синтезированных аудио
    temp_dir = os.path.join(NOTEBOOK_DIR, "temp_wer_audio")
    os.makedirs(temp_dir, exist_ok=True)
    
    # Обрабатываем примеры
    for i, original_text in enumerate(tqdm(test_texts, desc="Вычисление WER", unit="пример")):
        try:
            if not original_text or not original_text.strip():
                errors_count += 1
                continue
            
            # Синтезируем аудио
            temp_wav = os.path.join(temp_dir, f"synth_{i:03d}.wav")
            try:
                with wave.open(temp_wav, "wb") as wav_file:
                    voice.synthesize_wav(original_text, wav_file)
                
                # Проверяем, что файл создан и не пустой
                if not os.path.exists(temp_wav) or os.path.getsize(temp_wav) == 0:
                    errors_count += 1
                    if errors_count <= 3:
                        print(f"⚠️  Пример {i}: не удалось синтезировать аудио")
                    continue
            except Exception as synth_error:
                errors_count += 1
                if errors_count <= 3:
                    print(f"⚠️  Пример {i}: ошибка синтеза: {synth_error}")
                continue
            
            # Распознаем синтезированное аудио
            recognized_text = transcribe_audio_with_whisper(temp_wav)
            
            if not recognized_text:
                errors_count += 1
                if errors_count <= 3:
                    print(f"⚠️  Пример {i}: не удалось распознать аудио")
                continue
            
            # Вычисляем WER
            wer_value = calculate_wer(original_text, recognized_text)
            wers.append(wer_value)
            original_texts.append(original_text)
            recognized_texts.append(recognized_text)
            processed_count += 1
            
            # Выводим первые несколько примеров
            if processed_count <= 5:
                print(f"\nПример {processed_count}:")
                print(f"  Оригинальный текст: {original_text[:80]}...")
                print(f"  Распознанный текст: {recognized_text[:80]}...")
                print(f"  WER: {wer_value:.4f}")
        
        except Exception as e:
            errors_count += 1
            if errors_count <= 3:
                import traceback
                print(f"⚠️  Пример {i}: неожиданная ошибка: {e}")
                traceback.print_exc()
    
    # Очищаем временные файлы
    if os.path.exists(temp_dir):
        try:
            shutil.rmtree(temp_dir)
        except:
            pass
    
    # Выводим статистику
    print(f"\n{'='*60}")
    print("РЕЗУЛЬТАТЫ ВЫЧИСЛЕНИЯ WER ДЛЯ TTS")
    print(f"{'='*60}")
    print(f"\nСтатистика обработки:")
    print(f"  Всего примеров: {len(test_texts)}")
    print(f"  Успешно обработано: {processed_count}")
    print(f"  Ошибок: {errors_count}")
    
    if wers:
        print(f"\nWER (Word Error Rate) - чем меньше, тем лучше:")
        print(f"  Вычислено метрик: {len(wers)}")
        print(f"  Средний WER: {np.mean(wers):.4f}")
        print(f"  Медианный WER: {np.median(wers):.4f}")
        print(f"  Минимальный WER: {np.min(wers):.4f}")
        print(f"  Максимальный WER: {np.max(wers):.4f}")
        print(f"  Стандартное отклонение: {np.std(wers):.4f}")
        
        # Дополнительная статистика
        print(f"\nРаспределение WER:")
        print(f"  WER < 0.1 (отлично): {sum(1 for w in wers if w < 0.1)} ({100*sum(1 for w in wers if w < 0.1)/len(wers):.1f}%)")
        print(f"  WER < 0.2 (хорошо): {sum(1 for w in wers if w < 0.2)} ({100*sum(1 for w in wers if w < 0.2)/len(wers):.1f}%)")
        print(f"  WER < 0.3 (удовлетворительно): {sum(1 for w in wers if w < 0.3)} ({100*sum(1 for w in wers if w < 0.3)/len(wers):.1f}%)")
        print(f"  WER >= 0.3 (требует улучшения): {sum(1 for w in wers if w >= 0.3)} ({100*sum(1 for w in wers if w >= 0.3)/len(wers):.1f}%)")
        
        # Показываем примеры с лучшим и худшим WER
        if len(wers) > 0:
            best_idx = np.argmin(wers)
            worst_idx = np.argmax(wers)
            
            print(f"\nЛучший пример (WER = {wers[best_idx]:.4f}):")
            print(f"  Оригинал: {original_texts[best_idx][:100]}...")
            print(f"  Распознано: {recognized_texts[best_idx][:100]}...")
            
            print(f"\nХудший пример (WER = {wers[worst_idx]:.4f}):")
            print(f"  Оригинал: {original_texts[worst_idx][:100]}...")
            print(f"  Распознано: {recognized_texts[worst_idx][:100]}...")
        
        # Сохраняем результаты в файл
        results_file = os.path.join(NOTEBOOK_DIR, "wer_results.txt")
        with open(results_file, 'w', encoding='utf-8') as f:
            f.write("РЕЗУЛЬТАТЫ ВЫЧИСЛЕНИЯ WER ДЛЯ TTS МОДЕЛИ\n")
            f.write("="*60 + "\n\n")
            f.write(f"Модель: {onnx_model_path}\n")
            f.write(f"Обработано примеров: {processed_count}\n")
            f.write(f"Средний WER: {np.mean(wers):.4f}\n")
            f.write(f"Медианный WER: {np.median(wers):.4f}\n")
            f.write(f"Минимальный WER: {np.min(wers):.4f}\n")
            f.write(f"Максимальный WER: {np.max(wers):.4f}\n")
            f.write(f"Стандартное отклонение: {np.std(wers):.4f}\n\n")
            f.write("Детальные результаты:\n")
            f.write("-"*60 + "\n")
            for i, (orig, rec, wer_val) in enumerate(zip(original_texts, recognized_texts, wers)):
                f.write(f"\nПример {i+1} (WER = {wer_val:.4f}):\n")
                f.write(f"  Оригинал: {orig}\n")
                f.write(f"  Распознано: {rec}\n")
        
        print(f"\n✓ Результаты сохранены в: {results_file}")
    else:
        print(f"\n⚠️  Не удалось вычислить WER ни для одного примера.")
        print(f"   Возможные причины:")
        print(f"   - Ошибки при синтезе аудио")
        print(f"   - Ошибки при распознавании аудио")
        print(f"   - Проблемы с ASR моделью (Whisper)")

print(f"\n{'='*60}")


## Сравнение с базовой моделью из examples

Для оценки качества обученной модели сравним её с исходной базовой моделью из examples (примеры от преподавателя).


In [ ]:
print("="*60)
print("ЗАГРУЗКА БАЗОВОЙ МОДЕЛИ ИЗ EXAMPLES")
print("="*60)

# Ищем базовую модель в examples
base_model_path = os.path.join(NOTEBOOK_DIR, "examples", "nataha_ruslan.onnx")
base_config_path = os.path.join(NOTEBOOK_DIR, "examples", "nataha_ruslan.onnx.json")

# Если не найдена, ищем любую ONNX модель в examples
if not os.path.exists(base_model_path):
    examples_onnx = glob.glob(os.path.join(NOTEBOOK_DIR, "examples", "*.onnx"))
    if examples_onnx:
        base_model_path = examples_onnx[0]
        base_config_path = base_model_path + ".json"
        print(f"✓ Найдена базовая модель: {base_model_path}")
    else:
        print("⚠️  Базовая модель не найдена в examples")
        base_model_path = None
        base_config_path = None
else:
    print(f"✓ Найдена базовая модель: {base_model_path}")

# Загружаем базовую модель
base_voice = None
if base_model_path:
    try:
        if os.path.exists(base_config_path):
            print(f"Использование конфига: {base_config_path}")
            base_voice = PiperVoice.load(base_model_path, config_path=base_config_path, use_cuda=True)
        else:
            print("Загрузка без конфига...")
            base_voice = PiperVoice.load(base_model_path, use_cuda=True)
        print("✓ Базовая модель успешно загружена")
    except Exception as e:
        print(f"✗ Ошибка при загрузке базовой модели: {e}")
        import traceback
        traceback.print_exc()
        base_voice = None
else:
    print("⚠️  Не удалось найти базовую модель для сравнения")

print(f"\n{'='*60}")


In [ ]:
print("="*60)
print("ВЫЧИСЛЕНИЕ WER ДЛЯ БАЗОВОЙ МОДЕЛИ")
print("="*60)

# Проверяем, что базовая модель загружена и есть тестовые данные
if not base_voice:
    print("⚠️  Базовая модель не загружена. Пропускаем сравнение.")
    base_wers = []
    base_original_texts = []
    base_recognized_texts = []
elif 'test_texts' not in globals() or len(test_texts) == 0:
    print("⚠️  Тестовые данные не найдены. Пропускаем сравнение.")
    base_wers = []
    base_original_texts = []
    base_recognized_texts = []
else:
    print(f"\n✓ Базовая модель загружена")
    print(f"Обрабатываем {len(test_texts)} примеров для базовой модели...")
    
    base_wers = []
    base_original_texts = []
    base_recognized_texts = []
    base_processed_count = 0
    base_errors_count = 0
    
    # Создаем временную директорию для синтезированных аудио базовой модели
    temp_dir_base = os.path.join(NOTEBOOK_DIR, "temp_wer_audio_base")
    os.makedirs(temp_dir_base, exist_ok=True)
    
    # Обрабатываем примеры для базовой модели
    for i, original_text in enumerate(tqdm(test_texts, desc="WER базовая модель", unit="пример")):
        try:
            if not original_text or not original_text.strip():
                base_errors_count += 1
                continue
            
            # Синтезируем аудио базовой моделью
            temp_wav = os.path.join(temp_dir_base, f"synth_base_{i:03d}.wav")
            try:
                with wave.open(temp_wav, "wb") as wav_file:
                    base_voice.synthesize_wav(original_text, wav_file)
                
                if not os.path.exists(temp_wav) or os.path.getsize(temp_wav) == 0:
                    base_errors_count += 1
                    continue
            except Exception as synth_error:
                base_errors_count += 1
                continue
            
            # Распознаем синтезированное аудио
            recognized_text = transcribe_audio_with_whisper(temp_wav)
            
            if not recognized_text:
                base_errors_count += 1
                continue
            
            # Вычисляем WER
            wer_value = calculate_wer(original_text, recognized_text)
            base_wers.append(wer_value)
            base_original_texts.append(original_text)
            base_recognized_texts.append(recognized_text)
            base_processed_count += 1
        
        except Exception as e:
            base_errors_count += 1
    
    # Очищаем временные файлы
    if os.path.exists(temp_dir_base):
        try:
            shutil.rmtree(temp_dir_base)
        except:
            pass
    
    # Выводим статистику для базовой модели
    print(f"\n{'='*60}")
    print("РЕЗУЛЬТАТЫ WER ДЛЯ БАЗОВОЙ МОДЕЛИ")
    print(f"{'='*60}")
    print(f"\nСтатистика обработки:")
    print(f"  Всего примеров: {len(test_texts)}")
    print(f"  Успешно обработано: {base_processed_count}")
    print(f"  Ошибок: {base_errors_count}")
    
    if base_wers:
        print(f"\nWER базовой модели:")
        print(f"  Вычислено метрик: {len(base_wers)}")
        print(f"  Средний WER: {np.mean(base_wers):.4f}")
        print(f"  Медианный WER: {np.median(base_wers):.4f}")
        print(f"  Минимальный WER: {np.min(base_wers):.4f}")
        print(f"  Максимальный WER: {np.max(base_wers):.4f}")
        print(f"  Стандартное отклонение: {np.std(base_wers):.4f}")

print(f"\n{'='*60}")


In [ ]:
print("="*60)
print("СРАВНЕНИЕ МОДЕЛЕЙ")
print("="*60)

# Сравниваем результаты обеих моделей
if 'wers' in globals() and len(wers) > 0 and len(base_wers) > 0:
    print("\nСравнение обученной модели с базовой моделью:")
    print(f"{'='*60}\n")
    
    # Убеждаемся, что сравниваем одинаковое количество примеров
    min_len = min(len(wers), len(base_wers))
    if min_len > 0:
        trained_wers = wers[:min_len]
        base_wers_trimmed = base_wers[:min_len]
        
        trained_mean = np.mean(trained_wers)
        base_mean = np.mean(base_wers_trimmed)
        difference = trained_mean - base_mean
        improvement_pct = (base_mean - trained_mean) / base_mean * 100 if base_mean > 0 else 0
        
        print(f"Средний WER:")
        print(f"  Обученная модель: {trained_mean:.4f}")
        print(f"  Базовая модель:   {base_mean:.4f}")
        print(f"  Разница:          {difference:+.4f}")
        
        if difference < 0:
            print(f"  ✓ Обученная модель ЛУЧШЕ на {abs(improvement_pct):.2f}%")
        elif difference > 0:
            print(f"  ✗ Обученная модель ХУЖЕ на {abs(improvement_pct):.2f}%")
        else:
            print(f"  = Модели показывают одинаковый результат")
        
        print(f"\nМедианный WER:")
        trained_median = np.median(trained_wers)
        base_median = np.median(base_wers_trimmed)
        print(f"  Обученная модель: {trained_median:.4f}")
        print(f"  Базовая модель:   {base_median:.4f}")
        print(f"  Разница:          {trained_median - base_median:+.4f}")
        
        # Сравнение по примерам
        print(f"\nСравнение по примерам (из {min_len} общих):")
        better_count = sum(1 for t, b in zip(trained_wers, base_wers_trimmed) if t < b)
        worse_count = sum(1 for t, b in zip(trained_wers, base_wers_trimmed) if t > b)
        equal_count = sum(1 for t, b in zip(trained_wers, base_wers_trimmed) if t == b)
        
        print(f"  Обученная модель лучше: {better_count} ({100*better_count/min_len:.1f}%)")
        print(f"  Обученная модель хуже:  {worse_count} ({100*worse_count/min_len:.1f}%)")
        print(f"  Одинаковый результат:   {equal_count} ({100*equal_count/min_len:.1f}%)")
        
        # Сохраняем результаты сравнения
        comparison_file = os.path.join(NOTEBOOK_DIR, "wer_comparison.txt")
        with open(comparison_file, 'w', encoding='utf-8') as f:
            f.write("СРАВНЕНИЕ ОБУЧЕННОЙ И БАЗОВОЙ МОДЕЛЕЙ\n")
            f.write("="*60 + "\n\n")
            f.write(f"Обученная модель: {onnx_model_path}\n")
            f.write(f"Базовая модель: {base_model_path}\n\n")
            f.write(f"Средний WER:\n")
            f.write(f"  Обученная: {trained_mean:.4f}\n")
            f.write(f"  Базовая:   {base_mean:.4f}\n")
            f.write(f"  Разница:   {difference:+.4f} ({improvement_pct:+.2f}%)\n\n")
            f.write(f"Медианный WER:\n")
            f.write(f"  Обученная: {trained_median:.4f}\n")
            f.write(f"  Базовая:   {base_median:.4f}\n")
            f.write(f"  Разница:   {trained_median - base_median:+.4f}\n\n")
            f.write(f"Сравнение по примерам:\n")
            f.write(f"  Обученная лучше: {better_count} ({100*better_count/min_len:.1f}%)\n")
            f.write(f"  Обученная хуже:  {worse_count} ({100*worse_count/min_len:.1f}%)\n")
            f.write(f"  Одинаково:       {equal_count} ({100*equal_count/min_len:.1f}%)\n\n")
            f.write("Детальное сравнение по примерам:\n")
            f.write("-"*60 + "\n")
            for i, (orig, trained_wer, base_wer) in enumerate(zip(original_texts[:min_len], trained_wers, base_wers_trimmed)):
                diff = trained_wer - base_wer
                status = "лучше" if diff < 0 else "хуже" if diff > 0 else "равно"
                f.write(f"\nПример {i+1} (обученная {status} на {abs(diff):.4f}):\n")
                f.write(f"  Текст: {orig}\n")
                f.write(f"  WER обученной: {trained_wer:.4f}\n")
                f.write(f"  WER базовой:   {base_wer:.4f}\n")
        
        print(f"\n✓ Результаты сравнения сохранены в: {comparison_file}")
        
elif 'wers' not in globals() or len(wers) == 0:
    print("⚠️  Результаты WER для обученной модели не найдены.")
    print("   Сначала запустите ячейку 'Вычисление WER'")
elif len(base_wers) == 0:
    print("⚠️  Результаты WER для базовой модели не найдены.")
    print("   Сначала запустите ячейку 'Вычисление WER для базовой модели'")
else:
    print("⚠️  Недостаточно данных для сравнения")

print(f"\n{'='*60}")


In [ ]:
print("="*60)
print("ПРОСЛУШИВАНИЕ ПРИМЕРОВ")
print("="*60)

# Проверяем, что у нас есть результаты
if 'wers' in globals() and len(wers) > 0 and voice:
    from IPython.display import Audio, display
    
    # Находим лучший и худший примеры
    best_idx = np.argmin(wers)
    worst_idx = np.argmax(wers)
    
    # Создаем директорию для аудио примеров
    examples_audio_dir = os.path.join(NOTEBOOK_DIR, "wer_examples_audio")
    os.makedirs(examples_audio_dir, exist_ok=True)
    
    print(f"\nСинтез аудио для примеров...")
    
    # Лучший пример
    best_text = original_texts[best_idx]
    best_audio_path = os.path.join(examples_audio_dir, "best_example.wav")
    
    try:
        with wave.open(best_audio_path, "wb") as wav_file:
            voice.synthesize_wav(best_text, wav_file)
        print(f"✓ Лучший пример синтезирован: {best_audio_path}")
    except Exception as e:
        print(f"✗ Ошибка при синтезе лучшего примера: {e}")
        best_audio_path = None
    
    # Худший пример
    worst_text = original_texts[worst_idx]
    worst_audio_path = os.path.join(examples_audio_dir, "worst_example.wav")
    
    try:
        with wave.open(worst_audio_path, "wb") as wav_file:
            voice.synthesize_wav(worst_text, wav_file)
        print(f"✓ Худший пример синтезирован: {worst_audio_path}")
    except Exception as e:
        print(f"✗ Ошибка при синтезе худшего примера: {e}")
        worst_audio_path = None
    
    # Отображаем примеры для прослушивания
    print(f"\n{'='*60}")
    print("ЛУЧШИЙ ПРИМЕР (минимальный WER)")
    print(f"{'='*60}")
    print(f"WER: {wers[best_idx]:.4f}")
    print(f"Оригинальный текст: {best_text}")
    print(f"Распознанный текст: {recognized_texts[best_idx]}")
    
    if best_audio_path and os.path.exists(best_audio_path):
        print(f"\n🎵 Прослушать лучший пример:")
        display(Audio(filename=best_audio_path, autoplay=False))
    else:
        print("\n⚠️  Аудио файл не создан")
    
    print(f"\n{'='*60}")
    print("ХУДШИЙ ПРИМЕР (максимальный WER)")
    print(f"{'='*60}")
    print(f"WER: {wers[worst_idx]:.4f}")
    print(f"Оригинальный текст: {worst_text}")
    print(f"Распознанный текст: {recognized_texts[worst_idx]}")
    
    if worst_audio_path and os.path.exists(worst_audio_path):
        print(f"\n🎵 Прослушать худший пример:")
        display(Audio(filename=worst_audio_path, autoplay=False))
    else:
        print("\n⚠️  Аудио файл не создан")
    
    print(f"\n{'='*60}")
    print(f"Аудио файлы сохранены в: {examples_audio_dir}")
    print(f"  - Лучший пример: {best_audio_path}")
    print(f"  - Худший пример: {worst_audio_path}")
    print(f"{'='*60}")
    
elif 'wers' not in globals() or len(wers) == 0:
    print("⚠️  Результаты WER не найдены.")
    print("   Сначала запустите ячейку 'Вычисление WER'")
elif not voice:
    print("⚠️  TTS модель не загружена.")
    print("   Сначала загрузите модель в предыдущих ячейках")
else:
    print("⚠️  Недостаточно данных для отображения примеров")
